In [1]:
import sqlite3
import re
import os
import shutil
import sys
import datetime
import csv
import pandas as pd
from tqdm import tqdm
import numpy as np

#### Move all files to parent folder (gets rid of all sub folders)

In [2]:
#function to move through every subfolder and move csvs upwards in directory
def move_to_root_folder(root_path, cur_path):
    for filename in os.listdir(cur_path):
        if os.path.isfile(os.path.join(cur_path, filename)):
            shutil.move(os.path.join(cur_path, filename), os.path.join(root_path, filename))
        elif os.path.isdir(os.path.join(cur_path, filename)):
            move_to_root_folder(root_path, os.path.join(cur_path, filename))
        else:
            sys.exit("Should never reach here.")
    # remove empty folders
    if cur_path != root_path:
        os.rmdir(cur_path)
        
path = "C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\Jan_2010_Oct_2021" #path to parent folder (folder holding folders of csvs)
        
move_to_root_folder(path, path)

#### Removes all extra csv files

In [3]:
dir="C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\Jan_2010_Oct_2021" #path to folder with csvs

##### Removes all -outcome.csv and -stop-and-search.csv files

In [4]:
#os walk through folder and removes unnecessary data
for root, dirs, files in os.walk(dir):
    for name in files:
        if name.endswith(("-outcomes.csv")):
            os.remove(os.path.join(root, name))
        if name.endswith(("-stop-and-search.csv")):
            os.remove(os.path.join(root, name))

##### Removes all files from other police forces 

In [5]:
#os walk through folder and removes unnecessary data
for root, dirs, files in os.walk(dir):
    for name in files:
        if name.endswith(("-metropolitan-street.csv")):
            continue
        elif name.endswith(("-city-of-london-street.csv")):
            continue
        else:
            os.remove(os.path.join(root, name))

#### Inserts data from CVS into mySQL database

In [6]:
#connect to database
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database

#list names of all csv files
files = os.listdir('C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\Jan_2010_Oct_2021') #path to folder with csvs

#import csv files into database
for filename in files:
    with open ('C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\Jan_2010_Oct_2021\\' + filename, 'r') as f: #path to folder with csvs
        reader = csv.reader(f)
        columns = next(reader) 
        query = r'INSERT INTO Crime VALUES ({1})'
        query = query.format(','.join(columns), ','.join('?' * len(columns)))
        cursor = db_connect.cursor()
        for data in reader:
            cursor.execute(query, data)
        db_connect.commit()

#close connection to database (necessary, otherwise database locks)
cursor.close()
db_connect.close()

#### Removes 'Context' column

In [7]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database
cursor = db_connect.cursor()

query = """
    CREATE TEMPORARY TABLE TEMP (CrimeID, Month, ReportedBy, FallsWithin, Longitude, Latitude, Location, LSOACode, LSOAName, CrimeType, Outcome); 
    INSERT INTO TEMP SELECT CrimeID, Month, ReportedBy, FallsWithin, Longitude, Latitude, Location, LSOACode, LSOAName, CrimeType, Outcome FROM Crime; 
    DROP TABLE Crime;
    CREATE TABLE Crime (CrimeID, Month, ReportedBy, FallsWithin, Longitude, Latitude, Location, LSOACode, LSOAName, CrimeType, Outcome); 
    INSERT INTO Crime SELECT CrimeID, Month, ReportedBy, FallsWithin, Longitude, Latitude, Location, LSOACode, LSOAName, CrimeType, Outcome FROM TEMP;
    DROP TABLE TEMP; 
"""

cursor.executescript(query)

db_connect.commit()
cursor.close()
db_connect.close()

#### How to select according to time (example)

In [8]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database
cursor = db_connect.cursor()

#use "DATE(Month || '-01') BETWEEN '2019-01-01' AND '2020-01-01'" to select time range (inclusive range)
#database has dates stored as strings in Month column (disadvantage of python and sqlite3)
#concatenates '-01' to end of date strings to be able to parse as valid date string
#need to use DATE function for queries to read the strings as dates

query = """
    SELECT CrimeType, Month
    FROM Crime
    WHERE DATE(Month || '-01') BETWEEN '2019-01-01' AND '2020-01-01';
"""

df_example = pd.read_sql_query(query, db_connect)

cursor.close()
db_connect.close()

In [9]:
df_example

,CrimeType,Month
0,Anti-social behaviour,2019-01
1,Bicycle theft,2019-01
2,Other theft,2019-01
3,Anti-social behaviour,2019-01
4,Anti-social behaviour,2019-01
...,...,...
1209845,Violence and sexual offences,2020-01
1209846,Other crime,2020-01
1209847,Other crime,2020-01
1209848,Other crime,2020-01


#### Create new table 'MonthlyCrime' to aggregate totals per month

In [79]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database
cursor = db_connect.cursor()

years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

monthly_crime = pd.DataFrame({'Month':[], 'Total':[], 'Anti-social behaviour':[], 'Other crime':[], 'Violent crime':[], 'Burglary':[], 'Vehicle crime':[], 'Robbery':[]})

for year in years:
    for month in months:
        query = "SELECT * FROM Crime WHERE DATE(Month || '-01') BETWEEN '" + year + "-" + month + "-01' AND '" + year + "-" + month + "-01';"
        df_temp = pd.read_sql_query(query, db_connect)
        if len(df_temp) == 0:
            continue
        else:
            dict_temp = {}
            total = pd.DataFrame({'Month': [year + '-' + month], 'Total': [len(df_temp)]})
            crimetypes = df_temp['CrimeType'].value_counts().to_frame().transpose().reset_index(drop=True)
            data_temp = pd.concat([total, crimetypes], axis=1)
            monthly_crime = monthly_crime.append(data_temp, ignore_index=True)

cursor.close()
db_connect.close()

monthly_crime

,Month,Total,Anti-social behaviour,Other crime,Violent crime,Burglary,Vehicle crime,Robbery,Other theft,Criminal damage and arson,Drugs,Shoplifting,Public disorder and weapons,Violence and sexual offences,Theft from the person,Public order,Bicycle theft,Possession of weapons
0,2010-12,93077.0,34632.0,28489.0,11843.0,7905.0,7410.0,2798.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01,99795.0,33700.0,33009.0,12608.0,9041.0,8182.0,3255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-02,94857.0,32237.0,32238.0,11912.0,7794.0,7558.0,3118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-03,105681.0,35952.0,36582.0,13087.0,8330.0,8474.0,3256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-04,107721.0,41106.0,34191.0,13356.0,7627.0,8214.0,3227.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2021-08,89868.0,23415.0,780.0,NaN,4425.0,8857.0,1944.0,8308.0,4639.0,3308.0,3262.0,NaN,20243.0,3273.0,4937.0,2067.0,410.0
129,2021-09,91646.0,21993.0,890.0,NaN,4472.0,9214.0,2273.0,9098.0,4574.0,3088.0,2802.0,NaN,21755.0,3528.0,5309.0,2239.0,411.0
130,2021-10,94742.0,21093.0,982.0,NaN,4828.0,9377.0,2308.0,9538.0,4703.0,3780.0,3118.0,NaN,22528.0,4652.0,5343.0,2033.0,459.0
131,2021-11,90896.0,18064.0,891.0,NaN,4976.0,9616.0,2282.0,9872.0,4571.0,3748.0,3006.0,NaN,21793.0,4886.0,5020.0,1702.0,469.0


In [81]:
monthly_crime['Month'] = pd.to_datetime(monthly_crime['Month'], format="%Y-%m")
monthly_crime

,Month,Total,Anti-social behaviour,Other crime,Violent crime,Burglary,Vehicle crime,Robbery,Other theft,Criminal damage and arson,Drugs,Shoplifting,Public disorder and weapons,Violence and sexual offences,Theft from the person,Public order,Bicycle theft,Possession of weapons
0,2010-12-01,93077.0,34632.0,28489.0,11843.0,7905.0,7410.0,2798.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-01,99795.0,33700.0,33009.0,12608.0,9041.0,8182.0,3255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-02-01,94857.0,32237.0,32238.0,11912.0,7794.0,7558.0,3118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-03-01,105681.0,35952.0,36582.0,13087.0,8330.0,8474.0,3256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-04-01,107721.0,41106.0,34191.0,13356.0,7627.0,8214.0,3227.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2021-08-01,89868.0,23415.0,780.0,NaN,4425.0,8857.0,1944.0,8308.0,4639.0,3308.0,3262.0,NaN,20243.0,3273.0,4937.0,2067.0,410.0
129,2021-09-01,91646.0,21993.0,890.0,NaN,4472.0,9214.0,2273.0,9098.0,4574.0,3088.0,2802.0,NaN,21755.0,3528.0,5309.0,2239.0,411.0
130,2021-10-01,94742.0,21093.0,982.0,NaN,4828.0,9377.0,2308.0,9538.0,4703.0,3780.0,3118.0,NaN,22528.0,4652.0,5343.0,2033.0,459.0
131,2021-11-01,90896.0,18064.0,891.0,NaN,4976.0,9616.0,2282.0,9872.0,4571.0,3748.0,3006.0,NaN,21793.0,4886.0,5020.0,1702.0,469.0


In [82]:
monthly_crime.to_csv('monthlycrimedata', index=False)

##### Insert monthly crime data into new database table

In [83]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database

monthlycrimedata = 'C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\monthlycrimedata'

#delete any previous variable definition
del data

with open (monthlycrimedata, 'r') as f:
    reader = csv.reader(f)
    columns = next(reader) 
    query = r'INSERT INTO MonthlyCrime VALUES ({1})'
    query = query.format(','.join(columns), ','.join('?' * len(columns)))
    cursor = db_connect.cursor()
    for data in reader:
        cursor.execute(query, data)
    db_connect.commit()

cursor.close()
db_connect.close()

#### Mapping LSOAs to Local Authorities

In [11]:
local_authority_lookup = pd.read_csv('lsoa_localauthority_lookup.csv', encoding='latin-1', low_memory=False)

In [12]:
local_authority_lookup

,pcd7,pcd8,pcds,dointr,doterm,usertype,oa11cd,lsoa11cd,msoa11cd,ladcd,lsoa11nm,msoa11nm,ladnm,ladnmw
0,AB1 0AA,AB1 0AA,AB1 0AA,198001,199606.0,0,S00090303,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber Wes",Aberdeen City,NaN
1,AB1 0AB,AB1 0AB,AB1 0AB,198001,199606.0,0,S00090303,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber Wes",Aberdeen City,NaN
2,AB1 0AD,AB1 0AD,AB1 0AD,198001,199606.0,0,S00090399,S01006514,S02001237,S12000033,"Cults, Bieldside and Milltimber West - 02","Cults, Bieldside and Milltimber Wes",Aberdeen City,NaN
3,AB1 0AE,AB1 0AE,AB1 0AE,199402,199606.0,0,S00091322,S01006853,S02001296,S12000034,"Dunecht, Durris and Drumoak - 01","Dunecht, Durris and Drumoak",Aberdeenshire,NaN
4,AB1 0AF,AB1 0AF,AB1 0AF,199012,199207.0,1,S00090299,S01006511,S02001236,S12000033,Culter - 06,Culter,Aberdeen City,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640511,ZE3 9JW,ZE3 9JW,ZE3 9JW,198001,NaN,0,S00129022,S01012387,S02002322,S12000027,Shetland South - 01,Shetland South,Shetland Islands,NaN
2640512,ZE3 9JX,ZE3 9JX,ZE3 9JX,198001,NaN,0,S00129022,S01012387,S02002322,S12000027,Shetland South - 01,Shetland South,Shetland Islands,NaN
2640513,ZE3 9JY,ZE3 9JY,ZE3 9JY,198001,NaN,0,S00129025,S01012387,S02002322,S12000027,Shetland South - 01,Shetland South,Shetland Islands,NaN
2640514,ZE3 9JZ,ZE3 9JZ,ZE3 9JZ,198001,NaN,0,S00129025,S01012387,S02002322,S12000027,Shetland South - 01,Shetland South,Shetland Islands,NaN


In [30]:
la_names = ['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden', 'City of London', 'Croydon', 'Ealing', 'Enfield', 
            'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 
            'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 
            'Southwark','Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']
la_dict = dict()
for index, row in local_authority_lookup.iterrows():
    if row['ladnm'] in la_names:
        if row['lsoa11cd'] in la_dict:
            continue
        else:
            la_dict[row['lsoa11cd']] = row['ladnm']

In [32]:
len(la_dict)

4835

In [33]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database
cursor = db_connect.cursor()

#use "DATE(Month || '-01') BETWEEN '2019-01-01' AND '2020-01-01'" to select time range (inclusive range)
#database has dates stored as strings in Month column (disadvantage of python and sqlite3)
#concatenates '-01' to end of date strings to be able to parse as valid date string
#need to use DATE function for queries to read the strings as dates

query = """
    SELECT *
    FROM Crime;
"""

df_crime = pd.read_sql_query(query, db_connect)

cursor.close()
db_connect.close()

In [34]:
df_crime

,CrimeID,Month,ReportedBy,FallsWithin,Longitude,Latitude,Location,LSOACode,LSOAName,CrimeType,Outcome
0,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Vehicle crime,
1,,2010-12,City of London Police,City of London Police,-0.11494,51.518632,On or near Brownlow Street,E01000914,Camden 028B,Other crime,
2,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,
3,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,
4,,2010-12,City of London Police,City of London Police,-0.125742,51.516368,On or near Grape Street,E01000919,Camden 028D,Anti-social behaviour,
...,...,...,...,...,...,...,...,...,...,...,...
11879014,c0024565a0e65bf471d9962391ae0d891aa13a0065891d...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution
11879015,f84951d4c800d9a0a6f8c67a5f74811f4694a321f12aa6...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Investigation complete; no suspect identified
11879016,551c63c3773f1bc68646c6d0d830c6a06a91196b5e9381...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Under investigation
11879017,c874ec5ee338e645211e939dde3f535675e19dfc8cf67f...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution


In [39]:
la_list = []
for index, row in df_crime.iterrows():
    try:
        la_list.append(la_dict[row['LSOACode']])
    except:
        la_list.append(None)

In [41]:
len(la_list)

11879019

In [45]:
df_crime['LocalAuthority'] = la_list
df_crime

,CrimeID,Month,ReportedBy,FallsWithin,Longitude,Latitude,Location,LSOACode,LSOAName,CrimeType,Outcome,LocalAuthority
0,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Vehicle crime,,Camden
1,,2010-12,City of London Police,City of London Police,-0.11494,51.518632,On or near Brownlow Street,E01000914,Camden 028B,Other crime,,Camden
2,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,,Camden
3,,2010-12,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,,Camden
4,,2010-12,City of London Police,City of London Police,-0.125742,51.516368,On or near Grape Street,E01000919,Camden 028D,Anti-social behaviour,,Camden
...,...,...,...,...,...,...,...,...,...,...,...,...
11879014,c0024565a0e65bf471d9962391ae0d891aa13a0065891d...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution,None
11879015,f84951d4c800d9a0a6f8c67a5f74811f4694a321f12aa6...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Investigation complete; no suspect identified,None
11879016,551c63c3773f1bc68646c6d0d830c6a06a91196b5e9381...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Under investigation,None
11879017,c874ec5ee338e645211e939dde3f535675e19dfc8cf67f...,2021-12,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution,None


In [53]:
df_crime['Month'] = df_crime['Month'] + '-01'

In [54]:
df_crime

,CrimeID,Month,ReportedBy,FallsWithin,Longitude,Latitude,Location,LSOACode,LSOAName,CrimeType,Outcome,LocalAuthority
0,,2010-12-01,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Vehicle crime,,Camden
1,,2010-12-01,City of London Police,City of London Police,-0.11494,51.518632,On or near Brownlow Street,E01000914,Camden 028B,Other crime,,Camden
2,,2010-12-01,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,,Camden
3,,2010-12-01,City of London Police,City of London Police,-0.112952,51.518267,On or near Southampton Buildings,E01000914,Camden 028B,Other crime,,Camden
4,,2010-12-01,City of London Police,City of London Police,-0.125742,51.516368,On or near Grape Street,E01000919,Camden 028D,Anti-social behaviour,,Camden
...,...,...,...,...,...,...,...,...,...,...,...,...
11879014,c0024565a0e65bf471d9962391ae0d891aa13a0065891d...,2021-12-01,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution,None
11879015,f84951d4c800d9a0a6f8c67a5f74811f4694a321f12aa6...,2021-12-01,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Investigation complete; no suspect identified,None
11879016,551c63c3773f1bc68646c6d0d830c6a06a91196b5e9381...,2021-12-01,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Under investigation,None
11879017,c874ec5ee338e645211e939dde3f535675e19dfc8cf67f...,2021-12-01,Metropolitan Police Service,Metropolitan Police Service,,,No Location,,,Other crime,Local resolution,None


In [55]:
df_crime.to_csv('ukcrime', index=False)

In [64]:
db_connect = sqlite3.connect("C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\dc2-policeuk.db") #path to database

monthlycrimedata = 'C:\\Users\\darry\\Documents\\Y2Q3_Data-Challenge-2\\monthlycrimedata'

with open('ukcrime', 'r') as f:
    reader = csv.reader(f)
    columns = next(reader) 
    query = r'INSERT INTO Crime VALUES ({1})'
    query = query.format(','.join(columns), ','.join('?' * len(columns)))
    cursor = db_connect.cursor()
    for data in reader:
        cursor.execute(query, data)
    db_connect.commit()

cursor.close()
db_connect.close()